# INPUT & OUTPUT

In [1]:
##########
# INPUT  #
##########
final_suggestions='/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/files/final_suggestions.csv'

##########
# OUTPUT #
##########
output_df = '/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/top2vec/deep_learn_top2vec_topics_(only_rel_topics).csv'

# IMPORTS

In [2]:
from top2vec import Top2Vec
import pandas as pd

import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/repositories/tdmstudio-high-recall-information-retrieval-system/')
from utils import tdmstudio

# CODE

In [3]:
df = pd.read_csv(final_suggestions)

df['text'] = list(map(lambda id_: tdmstudio.get_title_and_text(tdmstudio.get_filename(str(id_))), df['id']))

df.head()



,Unnamed: 0,id,title,topic relevance,with relevant,has_labeled_as_relevant,is_labeled_relevant,new_score,text
0,0,1366203412,The birth of Toronto Style.,IR,False,True,False,0.100637,The birth of Toronto Style.\n \n ...
1,1,1412658808,ARCHITECTURE.,IR,False,True,False,0.076864,ARCHITECTURE.\n \n \n\n \n...
2,2,1371240574,STYLE SCUFFLE.,IR,False,True,False,0.077903,STYLE SCUFFLE.\n \n \n\n \...
3,3,1139223026,Calatrava's sculptural architecture.,IR,False,True,False,0.079527,Calatrava's sculptural architecture.\n ...
4,4,2606112512,Let's face it: Modernist architecture is history.,IR,False,True,False,0.084625,Let's face it: Modernist architecture is histo...


In [ ]:
print('Training model')
deep_model = Top2Vec(documents=list(df['text']), speed='deep-learn', workers=1)


2023-02-16 14:12:27,374 - top2vec - INFO - Pre-processing documents for training


Training model


2023-02-16 14:12:42,565 - top2vec - INFO - Creating joint document/word embedding


In [ ]:
print(f'Number of topics: {len(topic_nums)}')

In [ ]:
data = {'topic no.': [], 
        'size': [], 
        'relevant count': [], 
        'example': [], 
        'example(rel)':[], 
        'suggestions':[], 
        'relevants':[],
        'suggestions (by relevance)': [],
       }

topic_sizes, topic_nums = deep_model.get_topic_sizes()
topic_words, word_scores, topic_nums = deep_model.get_topics()

for word_ix,_ in enumerate(topic_words[0]):
    data[f'word_{word_ix:03}']=[]

df['top2vec']=deep_model.doc_top
df['doc_dist']=deep_model.doc_dist

for ix in range(len(topic_nums)):
    # Topic number & size 
    data['topic no.'].append(topic_nums[ix])
    data['size'].append(topic_sizes[ix])
    
    # auxdf
    auxdf=df[df['top2vec']==topic_nums[ix]]
    
    # Example highly relevant to topic
    data['example'].append('https://proquest.com/docview/'+str(auxdf.sort_values(by='doc_dist',ascending=False)['id'].iloc[0]))
    
    # Suggestions IDs (sorted by topic)
    suggestions_ids=auxdf[auxdf['relevant_or_suggested']=='sugg'].sort_values(by='doc_dist',ascending=False)['id']
    data['suggestions'].append(';'.join([str(elem) for elem in suggestions_ids]))
    
    # Suggestions IDs (by relevance)
    #NOT USED BECAUSE WE CANNOT RE RUN THIS SCRIPT, gives different topics 
    # and Serperi is already working with the other topics:
    
#     suggestions_ids=auxdf[auxdf['relevant_or_suggested']=='sugg'].sort_values(by='new_score',ascending=False)['id']
#     data['suggestions (by relevance)'].append(';'.join([str(elem) for elem in suggestions_ids]))
    
    # Labelled as relevant IDs
    relevant_ids=auxdf[auxdf['relevant_or_suggested']=='rel'].sort_values(by='doc_dist',ascending=False)['id']
    data['relevants'].append(';'.join([str(elem) for elem in relevant_ids]))
    
    
    # Filter auxdf to have only relevant (not suggestions)
    auxdf=auxdf[auxdf['relevant_or_suggested']=='rel']
    
    # Relevant example highly relevant to topic.
    if auxdf.shape[0]>0:
        data['example(rel)'].append('https://proquest.com/docview/'+str(auxdf.sort_values(by='doc_dist',ascending=False)['id'].iloc[0]))
    else:
        data['example(rel)'].append('')
        
    # No. of relevant in topic
    data['relevant count'].append(auxdf.shape[0])
    
    # Words in topic
    for word_ix,word in enumerate(topic_words[ix]):
        data[f'word_{word_ix:03}'].append(word)

output_df = pd.DataFrame(data)
output_df.to_csv(topics_csv)
output_df